In [1]:
%%capture

import warnings
# suppress user warnings during execution
warnings.filterwarnings(action='ignore', category=UserWarning)

%pip install --upgrade pip
%pip install spacy
%pip install ipywidgets
#%pip install -U jupyter
%sx python -m spacy download de_core_news_sm
%sx python -m spacy download en_core_web_sm
%sx python -m spacy download es_core_news_sm
%sx python -m spacy download fr_core_news_sm
%sx python -m spacy download it_core_news_sm
%sx python -m spacy download nl_core_news_sm
%sx python -m spacy download nb_core_news_sm
%sx python -m spacy download sv_core_news_sm

# suppress user warnings during execution
warnings.filterwarnings(action='ignore', category=UserWarning)


## Introduction
rematch2 performs specialised NER focussed on vocabularies and temporal entities, and implements a series of spaCy pipeline components to identify various entity types in free text, and create links to controlled vocabulary concepts. These components are used by the TemporalAnnotator class but may also be used directly for more fine-grained control if required. This notebook describes each component with working examples of usage. The following example python script uses the ``yearspan_ruler`` and ``periodo_ruler`` components to identify year span expressions (e.g. "Early 1900s") and named periods (terms from a specified Perio.do authority) in a given text. The output ia then displayed as inline HTML markup and as a textual tabular display.

In [ ]:
# Using a rematch2 component
import spacy
from spacy import displacy
from rematch2 import DocSummary, PeriodoRuler, YearSpanRuler, child_span_remover
from IPython.display import display, HTML

# use a predefined spaCy pipeline, disabling the default NER component
nlp = spacy.load("en_core_web_sm", disable = ['ner'])
# add rematch2 periodo_ruler component to the end of the pipeline
# looking for terms from Perio.do authority "p0kh9ds" (http://purl.org/heritagedata/schemes/eh_period)
nlp.add_pipe("yearspan_ruler", last=True)
nlp.add_pipe("periodo_ruler", last=True, config={
             "periodo_authority_id": "p0kh9ds"})
nlp.add_pipe("child_span_remover", last=True)
nlp.add_pipe("stop_list_span_remover", last=True, config={
             "stop_list": ["http://n2t.net/ark:/99152/p0kh9dsqkbq"]}) # not working??

# processing example text using the modified pipeline
# (text from https://doi.org/10.5284/1100106)
txt = '''
Eight trenches were excavated across the 0.93ha Site. Previous archaeological works immediately to the north of the \
Site in the early 1990s, including field walking and targeted excavation, recorded worked flint and a pit possibly dated to \
the Late Iron Age, while other investigations in the wider area have identified remains of Late Bronze Age, Early Iron Age and \
Saxon date. Evidence for truncation of the deposit sequence was noted across the development area, with the Site area potentially \
having been levelled and drained prior to the establishment of turf pitches, with topsoil present only as a very thin turf horizon overlying \
sterile subsoil and brickearth deposits. The evaluation identified a single pit of Late Bronze Age to Early Iron Age date. No other \
archaeological features were located during the works.
'''
doc = nlp(txt)

# display highlighted entities in context
html = DocSummary(doc).doctext_to_html()
display(HTML(html))
txt = DocSummary(doc).spans_to_text()
print(txt)

 start  end  token_start  token_end    label                                    id           text
   129  140           23         24 YEARSPAN                                          early 1990s
   245  258           43         45   PERIOD http://n2t.net/ark:/99152/p0kh9ds8p8k  Late Iron Age
   349  363           62         64   PERIOD http://n2t.net/ark:/99152/p0kh9dszskn Early Iron Age
   741  755          129        131   PERIOD http://n2t.net/ark:/99152/p0kh9dszskn Early Iron Age


The pipeline components contain language-specific patterns to facilitate similar processing in other languages. The following example again uses the ``yearspan_ruler`` and ``periodo_ruler`` components but with a French NER pipeline and a specific Perio.do authority file identifier (see http://n2t.net/ark:/99152/p02chr4 - "PACTOLS chronology periods used in DOLIA data, 2021").

In [ ]:
# Using a rematch2 component in another language
import spacy
#from spacy import displacy
from rematch2 import DocSummary, PeriodoRuler, YearSpanRuler, child_span_remover
from IPython.display import display, HTML

# use predefined (French) spaCy pipeline, disabling the default NER component
nlp = spacy.load("fr_core_news_sm", disable = ['ner'])
# add rematch2 component(s) to the end of the pipeline
nlp.add_pipe("yearspan_ruler", last=True)
nlp.add_pipe("periodo_ruler", last=True, config={"periodo_authority_id": "p02chr4"})
nlp.add_pipe("child_span_remover", last=True)
# process example text using the modified pipeline 
doc = nlp("Quelques éléments lithiques Würm IV dispersés sont des indicateurs de la période néolithique d'environ fin 11000 - début 10000 av JC. Un ensemble de fosses, circonscrit sur une surface de 35 m2, a livré du mobilier céramique du premier âge du Fer (Hallstatt C) et quelques éléments lithiques. Un bâtiment sur poteaux se situe à une distance de 100 m vers l’ouest. Pour la période de La Tène finale et gallo-romaine, l’ensemble des vestiges fossoyés et bâtis sont concentrés sur une parcelle à la croisée de deux chemins actuels présents sur le cadastre de 1807. L’élément structurant majeur est le fossé F6 qui traverse perpendiculairement la parcelle, large de 3 m pour une profondeur de 1,80 m sous la terre arable. Seul un angle de fossé, de nature différente et de plus petite taille, semble participer à cette même organisation du paysage. Un bâtiment de plan rectangulaire, 13 x 9 m, sur fondations de schiste dont deux angles ont été découverts, est orienté de façon identique au fossé F6. Il est très bien fondé sur une profondeur de 0,70 m avec de gros blocs de schiste. Si quelques rares tessons du Haut-Empire ont été trouvés dans la fondation du bâtiment, le mobilier céramique du colmatage de la zone humide située à 10 m au sud-ouest, est compris entre le milieu du Ier siècle et le début du IIe siècle de notre ère. Situé à 45 m plus au sud et parallèle au bâtiment, un fossé rectiligne de 3 m de large pour 1,80 m de profondeur sous la terre arable, scinde l’espace en deux. Son creusement en V à été comblé en deux temps. La première phase est une sédimentation naturelle qui a piégé quelques tessons protohistoriques et des scories ferreuses dont un culot de forge. Le colmatage supérieur est composé de matériaux issus de la démolition avec de très nombreuses tuiles, des blocs de schiste brut ainsi que quelques tessons de céramique gallo-romaine. De nombreux trous de poteaux et fosses se situent entre ces deux structures majeures. Un angle de fossé dessinant l’amorce d’un enclos se développe au sud. Du mobilier La Tène finale a également été trouvé dans une fosse située dans cette espace. Un réseau fossoyé se développe à l’est, très érodé du côté nord. Des fragments de céramique possiblement haut Moyen Âge ont été trouvés dans son comblement")
# display highlighted entities in context
html = DocSummary(doc).doctext_to_html()
display(HTML(html))
txt = DocSummary(doc).spans_to_text()
print(txt)

 start  end  token_start  token_end    label                                    id                             text
   103  132           15         21 YEARSPAN                                          fin 11000 - début 10000 av JC
  1261 1290          242        247 YEARSPAN                                          entre le milieu du Ier siècle
  1297 1329          250        256 YEARSPAN                                       début du IIe siècle de notre ère
    28   35            3          4   PERIOD http://n2t.net/ark:/99152/p02chr4gqd2                          Würm IV
    81   92           12         12   PERIOD http://n2t.net/ark:/99152/p02chr4ghc6                      néolithique
   228  246           42         45   PERIOD http://n2t.net/ark:/99152/p02chr4rx4z               premier âge du Fer
   248  257           47         47   PERIOD http://n2t.net/ark:/99152/p02chr4n5w5                        Hallstatt
   383  397           75         77   PERIOD http://n2t.net/ark:/99152/p

Other vocabulary-based components are included to identify terms from specific controlled vocabularies. The following example identifies terms originating from the Getty Art & Architecture Thesaurus and from the UK FISH cultural heritage vocabularies. As these are English language vocabularies they are used in association with an English NER pipeline.

In [ ]:
# Using vocabulary pipeline components
import spacy
from rematch2.VocabularyRuler import *
from IPython.display import display, HTML


txt = """
This collection comprises site data (images, a report, a project database and GIS data) from an archaeological excavation undertaken by Cotswold Archaeology between January and February 2020 at Lydney B Phase III, Archers Walk, Lydney, Gloucestershire. An area of 0.6ha was excavated within this phase (Phase III) of a wider development area.
Aside from three residual flints, none closely datable, the earliest remains comprised a small assemblage of Roman pottery and ceramic building material, also residual and most likely derived from a Roman farmstead found immediately to the north within the Phase II excavation area. A single sherd of Anglo-Saxon grass-tempered pottery was also residual.
The earliest features, which accounted for the majority of the remains on site, relate to medieval agricultural activity focused within a large enclosure. There was little to suggest domestic occupation within the site: the pottery assemblage was modest and well abraded, whilst charred plant remains were sparse, and, as with some metallurgical residues, point to waste disposal rather than the locations of processing or consumption. A focus of occupation within the Rodley Manor site, on higher ground 160m to the north-west, seems likely, with the currently site having lain beyond this and providing agricultural facilities, most likely corrals and pens for livestock. Animal bone was absent, but the damp, low-lying ground would have been best suited to cattle. An assemblage of medieval coins recovered from the subsoil during a metal detector survey may represent a dispersed hoard.
"""

# create pipeline and add one or more custom pipeline components
nlp = spacy.load("en_core_web_sm", disable=['ner'])
# AAT vocabulary pipeline components
#nlp.add_pipe("aat_activities_ruler", last=True)
# nlp.add_pipe("aat_agents_ruler", last=True)
# nlp.add_pipe("aat_associated_concepts_ruler", last=True)
# nlp.add_pipe("aat_materials_ruler", last=True)
# nlp.add_pipe("aat_objects_ruler", last=True)
# nlp.add_pipe("aat_physical_attributes_ruler", last=True)
# nlp.add_pipe("aat_styleperiods_ruler", last=True)
# FISH vocabulary pipeline components
nlp.add_pipe("fish_archobjects_ruler", last=True)
# nlp.add_pipe("fish_archsciences_ruler", last=True)
# nlp.add_pipe("fish_building_materials_ruler", last=True)
# nlp.add_pipe("fish_components_ruler", last=True)
# nlp.add_pipe("fish_event_types_ruler", last=True)
# nlp.add_pipe("fish_evidence_ruler", last=True)
# nlp.add_pipe("fish_maritime_craft_ruler", last=True)
nlp.add_pipe("fish_monument_types_ruler", last=True)
# nlp.add_pipe("fish_periods_ruler", last=True)
nlp.add_pipe("child_span_remover", last=True)
# process example text using the modified pipeline
doc = nlp(txt)

# display highlighted entities in context
html = DocSummary(doc).doctext_to_html()
display(HTML(html))
txt = DocSummary(doc).spans_to_text()
print(txt)


 start  end  token_start  token_end         label                                                           id          text
   439  449           82         82   FISH_OBJECT http://purl.org/heritagedata/schemes/mda_obj/concepts/141192    assemblage
   636  641          115        115   FISH_OBJECT http://purl.org/heritagedata/schemes/mda_obj/concepts/137051         sherd
   931  941          168        168   FISH_OBJECT http://purl.org/heritagedata/schemes/mda_obj/concepts/141192    assemblage
   986  999          177        178   FISH_OBJECT http://purl.org/heritagedata/schemes/mda_obj/concepts/100093 plant remains
  1045 1053          188        188   FISH_OBJECT http://purl.org/heritagedata/schemes/mda_obj/concepts/142906      residues
  1353 1357          244        244   FISH_OBJECT  http://purl.org/heritagedata/schemes/mda_obj/concepts/95390          pens
  1373 1384          248        249   FISH_OBJECT  http://purl.org/heritagedata/schemes/mda_obj/concepts/95074   Animal bone


The next example uses multiple components (``yearspan_ruler`` and ``periodo_ruler``) run against some pre-defined test text in different languages. A small user interface is included so you can change the selected language and re-run to observe the entities identified. 

In [ ]:
# Testing temporal components using any of the supported languages
import spacy
import json
from spacy import displacy
from rematch2 import YearSpanRuler, PeriodoRuler

#from test_examples_multilingual import test_examples_multilingual
import ipywidgets as widgets
from IPython.display import display, HTML

# load tests from JSON file
test_file_path = "./test_examples_multilingual.json"
with open(test_file_path, "r") as f:
    test_records = json.load(f)  

def run(btn):
    language = dropdown_language.value
    # get the (first) test present for the chosen language
    test = next(filter(lambda test: test["language"] == language, test_records), None)   
    if(test):
        # use predefined pipeline, disabling the default NER component
        nlp = spacy.load(test["pipe"], disable = ['ner'])

        # add the required pipeline component(s) to the end of the pipeline
        nlp.add_pipe("yearspan_ruler", last=True)
        nlp.add_pipe("periodo_ruler", last=True, config={"periodo_authority_id": test["periodo_authority_id"]})
        #print(nlp.pipe_names)
        
        # process some example text using the modified pipeline 
        doc = nlp(test["text"])
        
        # show highlighted entities in the example text
        output.clear_output(wait=True)
        with output:
            options = {
                "spans_key": "custom",
                "colors": { 
                    "PERIOD": "yellow", 
                    "YEARSPAN": "moccasin", 
                    "FISH_OBJECT": "plum",
                    "FISH_MONUMENT": "plum",
                    "FISH_ARCHSCIENCE": "lightpink",
                    "FISH_EVIDENCE": "aliceblue",
                    "FISH_MATERIAL": "antiquewhite",
                    "FISH_EVENTTYPE": "coral"
                }
            } 
            display(HTML(displacy.render(doc, style="span", options=options)))        
    else:
        with output:
            display(f"No tests for language '{language_code}'")

# define UI components
dropdown_language = widgets.Dropdown(
    options = [
        ["German", "de"], 
        ["English", "en"], 
        ["Spanish", "es"], 
        ["French", "fr"],
        ["Italian", "it"], 
        ["Dutch", "nl"],
        ["Norwegian", "no"],
        ["Swedish", "sv"] 
    ],
    value="en", 
    description='Language:',
    disabled=False
)

button_go = widgets.Button(description="Go")
input = widgets.HBox([dropdown_language, button_go])
output = widgets.Output(layout=widgets.Layout(overflow='scroll', border='1px solid black', height='250px'))
# what to do when the button is clicked..
button_go.on_click(run)
# display the UI components
display(input, output)

In [ ]:
"""
%load_ext autoreload
%autoreload 2

import json
import ipywidgets as widgets

# tell Python where the local modules are located
import sys
module_path = os.path.abspath(os.path.join('..', 'src'))
if module_path not in sys.path:
    sys.path.append(module_path)

# import local modules
from TemporalRecognizer import TemporalRecognizer
from PeriodoData import PeriodoData

# modules for UI
from IPython.display import display, HTML

testdata_file_name = "../src/test-examples.json"
testData = None 
with open(testdata_file_name, "r") as f:  # what if file doesn't exist?            
    testData = json.load(f)   

# get list of all Perio.do authorities
pd = PeriodoData(False) # create new instance, don't refresh cache
authorities = pd.get_authority_list() # returns [{id: "x", label: "label x"}, {id: "y", label: "label y"}]
authorities_for_dropdown = list(map(lambda x: [x["label"], x["id"]], authorities)) # dropdown requires array of tuples: [[label, id], [label, id]]

# define dropdown control for list of Perio.do authorities
dropdown_authorities = widgets.Dropdown(
    options = authorities_for_dropdown, 
    value=None, 
    description='Authority:',
    disabled=False
)

# define dropdown control for list of Perio.do periods
dropdown_periods = widgets.Dropdown(
    options = [], 
    value=None, 
    description='Periods:',
    disabled=False
)

# what to do when selected perio.do authority is changed
def on_dropdown_change(*args):
    periods = pd.get_period_list(dropdown_authorities.value)
    periods_for_dropdown = list(map(lambda item: [item["label"], item["id"]], periods))
    dropdown_periods.options = periods_for_dropdown

dropdown_authorities.observe(on_dropdown_change, 'value')

# define input textarea
#example_input_text_en = "the artefact was medieval or circa 1250-1275 or maybe post medieval? Underlying the modern made ground on the site was a layer covering the entire shaft area. This has been dated to c.1480-1800/1900 and interpreted as a post-medieval cultivation soil. Historic mapping illustrates that the site remained undeveloped through the post medieval period until the mid 19th century, when urban development around the site accelerated and construction of railways in this part of London began. On Gascoigne's 1703 map the site was open ground, the later maps of Rocque in 1746 and Horwood in 1799 show the area was in use as fields and Stanford's map of 1862 depicts the area surrounding Eleanor Street comprising of market gardens. These are all consistent with the archaeological evidence. Underlying the layer were natural terrace gravels. The archaeological fieldwork has demonstrated that remains relating to the Prehistoric, Roman or medieval period have not survived to the modern era, if they were once present on site."

textarea_input = widgets.Textarea(
    value="",
    placeholder='Type something',
    description='String:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='250px')    
)

# define language dropdown
dropdown_language = widgets.Dropdown(
    options = [
        ["German", "de"], 
        ["English", "en"], 
        ["Spanish", "es"], 
        ["French", "fr"],
        ["Italian", "it"], 
        ["Dutch", "nl"],
        ["Norwegian", "no"],
        ["Swedish", "sv"] 
    ],
    value="fr", 
    description='Language:',
    disabled=False
)

# define checkboxes for entity types
data = ["TEMPORAL", "MONUMENT", "MATERIAL", "EVENTTYPE", "ARCHSCIENCE"]
checkboxes = [widgets.Checkbox(value=False, description=label) for label in data]
entity_type_choices = widgets.VBox(children=checkboxes)

#input_text = ""
for testItem in testData:
        if testItem["language"] == dropdown_language.value:
            textarea_input.value = testItem["value"]  
            break       

# define output format dropdown
dropdown_output_format = widgets.Dropdown(
    options = ["html", "json", "csv", "tsv"], 
    value="html", 
    description='Format:',
    disabled=False
)

output = widgets.Output(
    layout=widgets.Layout(overflow='scroll', border='1px solid black', height='250px')
)

def on_button_go_clicked(b):    
    input_text = textarea_input.value
    language = dropdown_language.value
    output_format = dropdown_output_format.value
    authority_id = dropdown_authorities.value
    tr = TemporalRecognizer(language, authority_id)
    entities = tr.get_entities(input_text, output_format)
    #formatted = TemporalRecognizer.format_entities(input_text, entities, output_format)
    #print(formatted)
    output.clear_output(wait=True)
    with output:
        #pprint(formatted)        
        display(HTML(entities))
        #display(HTML('<a href="http://example.com">link</a>'))
    #display(formatted)
        
# define 'Go' button and output area
button_go = widgets.Button(description="Go")
button_go.on_click(on_button_go_clicked)
    
# display all controls on the screen

display(entity_type_choices)
display(dropdown_authorities)
#display(dropdown_periods)
display(dropdown_language)
display(dropdown_output_format)
display(textarea_input)
display(button_go)
display(output) 
#display(HTML('<a href="http://example.com">link</a>'))
"""